## Introduction

This Jupyter notebook is a tool to evaluate the consistency of ML Test Evaluation performed by ChatGPT based on the research paper (Alexander, R., Katz, L., Moore, C., & Schwartz, Z. (2023)). \
It serves the purpose of evaluating the application performance before and after checklist modification, and evaluating the application performance upon model setting changes.

### Libraries

In [105]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import DirectoryLoader, PythonLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain

import itertools
import json
import pandas as pd

# Prepare .env and API Key before running the script below
load_dotenv()

True

## Data

Please specify the `test_functions_directory` below to load the ML test code base for the evaluation.\
The loaded test functions will be further split.

In [106]:
test_functions_directory = '../../../lightfm/tests'

In [107]:
# load doc
loader = DirectoryLoader(
    path = test_functions_directory, 
    glob="**/*.py", 
    show_progress=True, 
    #use_multithreading=True,
    loader_cls=PythonLoader
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(docs)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 1078.11it/s]


## Parameters

Please specify the parameters, e.g. checklist, to be evaluated

In [188]:
# temperatures = [0, 0.2, 0.5, 1]
temperatures = [0.1]
models = ['gpt-3.5-turbo']
roles = ['a senior machine learning engineer who specializes in performing Machine Learning system testing']

In [170]:
checklist_real_before = '''
2.1: Verify the function for loading data files load the file if the files exists with the right format, and doesn't load the file if it doesn't exist, and that it returns the expected results.
2.2: Verify the functions for saving data and figures can write as expected. They should check the if the write operation is successfully carried out, and the content is in an expected format.
3.1: Ensure that all data files are non-empty and contain the necessary data to proceed with the analysis or processing tasks.
3.2: Check that the data to be ingested is in the format expected by the processing algorithms (e.g., Is the CSV loaded as a `pd.DataFrame`? Is the image file loaded as a `np.array`, or a `PIL.Image`?) and that their structure matches the expected schema, any present.
'''

In [171]:
checklist_real_after = """
2.1: Ensure that data-loading functions correctly load files when they exist and match the expected format, handle non-existent files appropriately, and return the expected results.
2.2: Verify that functions for saving data and figures perform write operations correctly, checking that the operation succeeds and the content matches the expected format.
3.1: Ensure all data files are non-empty and contain the necessary data required for further analysis or processing tasks.
3.2: Verify that the data to be ingested matches the format expected by processing algorithms (like pd.DataFrame for CSVs or np.array for images) and adheres to the expected schema.
"""

In [258]:
checklists = [checklist_real_before] # , checklist_real_after

In [259]:
human_message = """
Your task is to answer each question in the checklist using only the provided test functions. Do not disclose your work for this step.
Then, decide the completion score in a fraction format based on your answers.
Desired JSON format:
{
    "Checklist Evaluation":
        "ID": 
        "Requirement": -||-
        "Evaluation": Satisfied/Partially Satisfied/Not Satisfied
    "Completeness Score": 
        "Number of satisfied requirements":
        "Number of partially satisfied requirements":
        "Number of not satisfied requirements":
        "Number of requirements":
}
"""

In [260]:
user_prompts = [human_message]

In [270]:
params = [
    {
        'Param_Set_ID': i,
        'temperature': item[0],
        'model': item[1],
        'role': item[2],
        'checklist': item[3],
        'user_prompt': item[4],
    } for i, item in enumerate(itertools.product(
        temperatures, 
        models,
        roles,
        checklists,
        user_prompts,
    ))
]

In [271]:
pd.DataFrame(params)

,Param_Set_ID,temperature,model,role,checklist,user_prompt
0,0,0.1,gpt-3.5-turbo,a senior machine learning engineer who special...,\n2.1: Verify the function for loading data fi...,\nYour task is to answer each question in the ...


## API Running

Incorporate the data, prompts and parameters, feed into OpenAI API for test runs and fetch responses.

In [272]:
num_test_runs = 5

In [273]:
def extract_json(response, start='{', end='}'):
    start_idx = response.index(start)
    end_idx = response[::-1].index(end)
    if end_idx == 0:
        string = response[start_idx:]
    else:
        string = response[start_idx:-end_idx]
    return json.loads(string)

In [274]:
results = []
for param in params:
    chat = ChatOpenAI(model=param['model'], 
                      temperature=param['temperature'],
                     )
    
    # define prompt and chat
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"You are {param['role']}." + "Extract and analyze the test functions from the codes:\n\n{context}"),
        ("system", f"Here is the Machine Learning system testing checklist delimited by triple quotes '''{param['checklist']}'''"),
        MessagesPlaceholder(variable_name="messages")
    ])
    
    # combine prompt, chat and doc
    chain = create_stuff_documents_chain(chat, prompt) 

    # evaluation test run
    for i in range(num_test_runs):
        result = dict()
        report = chain.invoke({
            "context": all_splits,
            "messages": [
                HumanMessage(content=param['user_prompt'])
            ],
        })

        result['report'] = report
        result['Param_Set_ID'] = param['Param_Set_ID']
        result['Test_No'] = i+1
        results.append(result)

In [275]:
for i in range(len(results)):
    results[i]['report'] = extract_json(results[i]['report'])

In [298]:
# results[2]['report']

In [277]:
results_df = pd.DataFrame(results)
results_df = pd.concat([results_df, results_df['report'].apply(pd.Series)], axis=1)
results_df = results_df.drop(columns='report')
results_df

,Param_Set_ID,Test_No,Checklist Evaluation,Completeness Score
0,0,1,"[{'ID': '2.1', 'Requirement': 'Verify the func...","{'Number of satisfied requirements': 3, 'Numbe..."
1,0,2,"[{'ID': '2.1', 'Requirement': 'Verify the func...","{'Number of satisfied requirements': 3, 'Numbe..."
2,0,3,"[{'ID': '2.1', 'Requirement': 'Verify the func...","{'Number of satisfied requirements': 3, 'Numbe..."
3,0,4,"[{'ID': '2.1', 'Requirement': 'Verify the func...","{'Number of satisfied requirements': 3, 'Numbe..."
4,0,5,"[{'ID': '2.1', 'Requirement': 'Verify the func...","{'Number of satisfied requirements': 2, 'Numbe..."


## Result & Evaluation

The evaluation will be based on 2 metrics calculated from the response:
- Completeness Score distribution: The distribution of the `num_test_runs` completeness scores per each set of parameters
- Consistency Score: Out of all `checklist` items, the proportion of results remain consistent among `num_test_runs` runs per each set of parameters

In [296]:
completeness_score_df = results_df.drop(columns='Checklist Evaluation')
completeness_score_df = pd.concat([completeness_score_df, completeness_score_df['Completeness Score'].apply(pd.Series)], axis=1)
completeness_score_df['completeness_score'] = completeness_score_df['Number of satisfied requirements'] / completeness_score_df['Number of requirements']
completeness_score_df = completeness_score_df.pivot(index='Test_No', columns='Param_Set_ID', values='completeness_score')
# completeness_score_df.reset_index()
# completeness_score_df.columns.name = 'completeness_score'

In [297]:
completeness_score_df.T

Test_No,1,2,3,4,5
Param_Set_ID,,,,,
0,0.75,0.75,0.75,0.75,0.5


In [304]:
consistency_df = results_df.drop(columns='Completeness Score')
consistency_df = consistency_df.explode('Checklist Evaluation')
consistency_df = pd.concat([consistency_df, consistency_df['Checklist Evaluation'].apply(pd.Series)], axis=1)
consistency_df = consistency_df.pivot(index=['Param_Set_ID', 'ID'], columns=['Test_No'], values=['Evaluation'])['Evaluation']
consistency_df['consistency'] = consistency_df.eq(consistency_df.iloc[:, 0], axis=0).all(1)
# consistency_df = consistency_df.reset_index().drop(columns=['Param_Set_ID'])
consistency_df.columns.name = None

In [305]:
consistency_df

1                    2  \
Param_Set_ID ID                                              
0            2.1            Satisfied            Satisfied   
             2.2  Partially Satisfied        Not Satisfied   
             3.1            Satisfied            Satisfied   
             3.2  Partially Satisfied  Partially Satisfied   

                                    3                    4  \
Param_Set_ID ID                                              
0            2.1            Satisfied            Satisfied   
             2.2        Not Satisfied            Satisfied   
             3.1            Satisfied            Satisfied   
             3.2  Partially Satisfied  Partially Satisfied   

                                    5  consistency  
Param_Set_ID ID                                     
0            2.1            Satisfied         True  
             2.2        Not Satisfied        False  
             3.1            Satisfied         True  
             3.2  Partially Satisfied         True

In [303]:
# consistency_df = consistency_df.reset_index().rename(columns={"ID": "Checklist_ID"})
# consistency_df = consistency_df.groupby(['Param_Set_ID']).mean('consistency')
# consistency_df